In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [84]:
#importing train dataset
df = pd.read_csv("train.csv")

In [85]:
#df.info()

All the entries for the train.csv are non-null values.
However, there might be NONE entries.

In [86]:
#df_test = pd.read_csv("test.csv")
#df_test.info()

All the entries for the test.csv are non-null values.

In [87]:
#making list of data fields for test and train data
#f_test = ['Id', 'Dates', 'DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y']
#f = f_test + ['Category', 'Descript', 'Resolution']

In [88]:
df.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [89]:
 def extract_date(df):
    """
    function specifically for Dates feature only
    creates new features 
    Year  YYYY
    Month MM
    Date  DD
    Time  HH:MM:SS
    """
    df['Year'] = df['Dates'].apply(lambda x: x[:4])
    df['Month'] = df['Dates'].apply(lambda x: x[5:7])
    df['Date'] = df['Dates'].apply(lambda x: x[8:10])
    df['Time'] = df['Dates'].apply(lambda x: x[-9:])
    return

def make_binary_features(df, feature):
    """
    creates new features with feature name as the name of data 
    if the original data match the new feature name, the data will be 1
    if the original data does not match the new feature name, the data will be 0

    
    ex 
    make_binary_features(df, 'DayOfWeek')
    will create new features
    Monday, Tuesday, Wednesday, Thursday, Friday, Saturday and Sunday
    where
    df['Monday'] will have value 1 for all Mondays and 0 for the rest
    """
    for new_feature in df[feature].unique():
        df[new_feature] = df[feature]
        df.loc[df[new_feature] != new_feature, new_feature] = 0
        df.loc[df[new_feature] == new_feature, new_feature] = 1
    return

#days = df[f[2]].unique() #7 days
#pdds = df[f[3]].unique() #10 pdds
#categories = df[f[7]].unique() #39 categories
#descriptions = df[f[8]].unique() #879 descriptions
#resolutions = df[f[9]].unique() #17 resolutions

extract_date (df)
make_binary_features(df, 'DayOfWeek')
#make_binary_features(df, 'PdDistrict')
df.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,Year,Month,Date,Time,Wednesday,Tuesday,Monday,Sunday,Saturday,Friday,Thursday
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,2015,05,13,23:53:00,1,0,0,0,0,0,0
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,2015,05,13,23:53:00,1,0,0,0,0,0,0
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414,2015,05,13,23:33:00,1,0,0,0,0,0,0
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873,2015,05,13,23:30:00,1,0,0,0,0,0,0
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541,2015,05,13,23:30:00,1,0,0,0,0,0,0
